# Interact with fission product yields in ENDF-6 format (MF=8, MT=454/459)

First let's import `sandy` and the other python packages for formatting and postprocessing that are used in this notebook.

In [1]:
import yaml
import urllib
import os

import pandas as pd

In [2]:
import sandy

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_context("notebook")
sns.set_style("whitegrid")

%matplotlib inline

In [4]:
%load_ext autoreload
%autoreload 2

## Parsing the data in ENDF-6 format

The first thing that we want to do with an evaluated nuclear data  file in ENDF-6 format is to load its content into python using `sandy`.
By calling `sandy.Endf6.from_file`, the ENDF-6 file is parsed and its content is split into text sections, each of them indexed by a unique combination of MAT, MF and MT numbers. 

Below we report an example using the JEFF-3.3 fission yield evaluated file available on the [OECD/NEA website](https://www.oecd-nea.org/dbdata/jeff/jeff33/index.html).
The file content is loaded into the `Endf6` instance that we name `tape`.

In [5]:
url = "https://www.oecd-nea.org/dbdata/jeff/jeff33/downloads/JEFF33-nfy.asc"
with urllib.request.urlopen(url) as f:
    text_fy = f.read().decode('utf-8')

In [6]:
%%capture
tape = sandy.Endf6.from_text(text_fy)

In [7]:
tape

MAT   MF  MT 
9040  1   451     9.023200+4 2.300450+2         -1          1  ...
      8   454     9.023200+4 2.300450+2          2          0  ...
          459     9.023200+4 2.300450+2          2          0  ...
9222  1   451     9.223300+4 2.310380+2         -1          1  ...
      8   454     9.223300+4 2.310380+2          3          0  ...
          459     9.223300+4 2.310380+2          3          0  ...
9225  1   451     9.223400+4 2.320300+2         -1          1  ...
      8   454     9.223400+4 2.320300+2          1          0  ...
          459     9.223400+4 2.320300+2          1          0  ...
9228  1   451     9.223500+4 2.330250+2         -1          1  ...
      8   454     9.223500+4 2.330250+2          3          0  ...
          459     9.223500+4 2.330250+2          3          0  ...
9231  1   451     9.223600+4 2.340180+2         -1          1  ...
      8   454     9.223600+4 2.340180+2          2          0  ...
          459     9.223600+4 2.340180+2         

Now, the information available in each `(MAT,MF,MT)` section in ASCII format can be extracted using method `.read_section`, 
as shown below.

> Recall that MF=8 MT=454 are the indices for independent fission yields, MF=8 MT=459 for cumulative fission yields

In [8]:
mat = 9437  # This is Pu-239
mf = 8
mt = 454

section = tape.read_section(mat, mf, mt)

For a better visualization of the FY data structure generated by `.read_section` we resort to the `yaml` package and we print only the first 11 lines.

In [9]:
print("\n".join(yaml.dump(section).splitlines()[:11]))

AWR: 236.999
E:
  0.0253:
    INTERP: 1
    ZAP:
      10010:
        DFY: 7.052699999999999e-06
        FY: 4.08e-05
      10020:
        DFY: 2.8929000000000003e-06
        FY: 1.347e-05


The data contained in the ENDF-6 section are stored in nested dictionaries, 
according to the following structure that tries and replicate the ENDF-6 hierarchy.

```yaml
    AWR : atomic weight ratio
    E :   # dictionary of tabulated energies at which fission yields are given
        energy :  # energy of the neutron causing fission (eV)
            INTERP : interpolation scheme between this and the following energy point
            ZAP :   # dictionary of fission products
                Z*1000 + A*10 + M :
                    FY : fission yield (fraction)
                    DFY : uncertainty on fission yield (same unit as FY)
                Z2*1000 + A2*10 + M2 : ...
        energy_2 : ...
```

## Create a `sandy.Fy` object

Class `sandy.Fy` was developed to store and process fission yield data.

The ASCII content in a `sandy.Endf6` instance can be directly imported into a `sandy.Fy` instance using 
classmethod `.from_endf6`, where the `sandy.Endf6` object is passed as an argument.

In [ ]:
fy = sandy.Fy.from_endf6(tape)
fy

The `pd.DataFrame` containing the source data is stored in attribute `.data`, which contains fission yields FY together with the corresponding:
- MAT
- MT
- ZAM
- ZAP
- E

Here ZAM and ZAP are the `Z*10000 + A*10 + M` representation for the parent and daugther nuclides.

## How to filter the fission yield data 

The fission yield data can be filtered directly applying the `pandas` functionality to attribute `.data`.
However, we recommend using method `.filter_by` to make sure that the returned object be still a `sandy.Fy` instance.

Below are two examples where `sandy.Fy` instances are created first by filtering only the cumulative yield at thermal energy for the fission of U-235 (`ZAM=922350`), and then keeping only the cumulative fission yields at thermal energy for Cs-137 (`ZAP=551370`) independently of the fissioning nuclide.

In [ ]:
cfyU235 = fy.filter_by("ZAM", 922350).filter_by("MT", 459).filter_by("E", 2.53e-2)
cfyU235

In [ ]:
cfyCs137 = fy.filter_by("ZAP", 551370).filter_by("MT", 459).filter_by("E", 2.53e-2)
cfyCs137

## Plotting

Plotting fission yields as a function of mass or charge can be cumbersome with the current information available in a `sandy.Fy` instance.
In fact, to do so columns ZAM or ZAP should be *expanded* to their source components Z, A, M, where Z is the charge number, A is the number of nucleons and M is the metastate number.

Here we introduce two new methods that do exactly what required: `._expand_zap` and `._expand_zam`.

The first method returns a copy a of the `.data` attribute with three extra columns: the Z, A and M of the fission product.
The second method returns a copy a of the `.data` attribute with three extra columns: the Z, A and M of the fissioning nuclide.

> Methods `._expand_zap` and `._expand_zam` return a `pd.DataFrame` and not a `sandy.Fy` instance.

In [ ]:
cfyU235._expand_zap().head()

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10, 5))
ax[0] = sns.scatterplot(x="A", y="FY", data=cfyU235._expand_zap(), ax=ax[0])
ax[1] = sns.scatterplot(x="Z", y="FY", data=cfyU235._expand_zap(), ax=ax[1])

ax[0].set_title("by mass")
ax[1].set_title("by charge")

plt.tight_layout();

In [ ]:
cfyCs137._expand_zam().head()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5), sharey=True)

ax[0] = sns.scatterplot(x="A", y="FY", data=cfyCs137._expand_zam(), ax=ax[0])
ax[0].set_title("by mass")

ax[1] = sns.scatterplot(x="Z", y="FY", data=cfyCs137._expand_zam(), ax=ax[1])
ax[1].set_title("by charge")

plt.tight_layout();

## Tabulated fission yields as a function of energy

It is sometimes important to have access to the fission yield data (either cumulaitve or independent, not both) as a tabulated function of energy.
This can be done by calling method `.energy_table` where the fissioning nuclide is specified as an argument either as a ZAM or MAT number. 

Below we report an example for the cumulative yields of U-235.

In [ ]:
fy.energy_table(922350, by="ZAM", kind="cumulative")

## Write to HDF5 file

Codes like ALEPH read fission yields from nuclear data files in HDF5 format.
To produce such files we can use `sandy` and method `Fy.to_hdf5`, which extracts and stores energy tables for each set of fission yields found in the `Fy` object.

As an example, we can transfer the whole content of the `fy` instance to a HDF5 file `'fy.hdf5'`.

In [ ]:
file = 'fy.hdf5'
if os.path.exists(file):
    os.remove(file)
fy.to_hdf5(file, "jeff_33");

The group key for each set of fission yields is `"library/fy/kind/zam"`, where
- library: is the lowercase name of the library
- fy: is the key "fy"
- kind: is either "independent" or "cumulative"
- zam: is the ZAM number of the parent nuclide

> The energy values in the HDF5 file are converted into MeV.

In [ ]:
pd.read_hdf(file, key='jeff_33/fy/cumulative/922380')

## Apply custom perturbations

to be written